In [1]:
import time
from options.train_options import TrainOptions
from data import CreateDataLoader , find_dataset_using_name
from models import create_model
from util.util import confusion_matrix, getScores, tensor2labelim, tensor2im, print_current_losses
import numpy as np
import random
import torch
import cv2

## Seed Every Thing 


In [2]:
    
seed  = 20
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [3]:
train_opt = { 'batch_size': 2,
                    'beta1': 0.5,
                    'dataroot':"./datasets/kitti/",
          'checkpoints_dir': './checkpoints',
           'continue_train': False,
                  'dataset': 'kitti',
                    'epoch': 'latest',
              'epoch_count': 1,
                  'gpu_ids': 0,
                'init_gain': 0.02,
                'init_type': 'kaiming',
                  'isTrain': True,                       
                'lambda_L1': 100.0,
                       'lr': 0.001,
          'lr_decay_epochs': 25,
           'lr_decay_iters': 5000000,
                 'lr_gamma': 0.9,
                'lr_policy': 'lambda',
                    'model': 'roadseg',
                 'momentum': 0.9,
                     'name': "exp1",
                   'nepoch': 1000,
                     'norm': 'instance',
              'num_threads': 8,
                    'phase': 'train',
               'print_freq': 10,
                     "seed":0,
           'serial_batches':False,
                'useHeight':384,
                 'useWidth':1248,
                  "use_sne":False,
                  'verbose':False,
             'weight_decay': 0.0005,
             "gpu_ids":[0]
            
}


class Options:
  def __init__(self,**kwargs):
    self.dataroot = kwargs.get('dataroot')
    self.batch_size = kwargs.get('batch_size')
    self.beta1 = kwargs.get('beta1')
    self.checkpoints_dir = kwargs.get('checkpoints_dir')
    self.continue_train = kwargs.get('continue_train')
    self.dataset = kwargs.get('dataset')
    self.epoch = kwargs.get('epoch')
    self.epoch_count = kwargs.get('epoch_count')
    self.gpu_ids = kwargs.get('gpu_ids')
    self.init_gain = kwargs.get('init_gain')
    self.init_type = kwargs.get('init_type')
    self.isTrain = kwargs.get('isTrain')
    self.lambda_L1 = kwargs.get('lambda_L1')
    self.lr = kwargs.get('lr')
    self.lr_decay_iters = kwargs.get('lr_decay_iters')
    self.lr_decay_epochs = kwargs.get('lr_decay_epochs')
    self.lr_gamma = kwargs.get('lr_gamma')
    self.lr_policy = kwargs.get('lr_policy')
    self.model  = kwargs.get('model')
    self.use_sne = kwargs.get('use_sne')
    self.momentum = kwargs.get('momentum')
    self.name = kwargs.get('name')
    self.nepoch = kwargs.get('nepoch')
    self.norm = kwargs.get('norm')
    self.num_threads = kwargs.get('num_threads')
    self.phase = kwargs.get('phase')
    self.print_freq = kwargs.get('print_freq')
    self.seed = kwargs.get('seed')
    self.serial_batches = kwargs.get('serial_batches')
    self.useHeight = kwargs.get('useHeight')
    self.useWidth = kwargs.get('useWidth')
    self.gpu_ids = kwargs.get('gpu_ids')
    self.weight_decay = kwargs.get('weight_decay')
    self.verbose = kwargs.get('verbose')


In [4]:
opt = Options(**train_opt)
train_opt = opt

In [5]:
train_opt.nepoch = 5

In [6]:
import os.path
import torchvision.transforms as transforms
import torch
import cv2
import numpy as np
import glob
from data.base_dataset import BaseDataset
from models.sne_model import SNE


class kittiCalibInfo():
    """
    Read calibration files in the kitti dataset,
    we need to use the intrinsic parameter of the cam2
    """
    def __init__(self, filepath):
        """
        Args:
            filepath ([str]): calibration file path (AAA.txt)
        """
        self.data = self._load_calib(filepath)

    def get_cam_param(self):
        """
        Returns:
            [numpy.array]: intrinsic parameter of the cam2
        """
        return self.data['P2']

    def _load_calib(self, filepath):
        rawdata = self._read_calib_file(filepath)
        data = {}
        P0 = np.reshape(rawdata['P0'], (3,4))
        P1 = np.reshape(rawdata['P1'], (3,4))
        P2 = np.reshape(rawdata['P2'], (3,4))
        P3 = np.reshape(rawdata['P3'], (3,4))
        R0_rect = np.reshape(rawdata['R0_rect'], (3,3))
        Tr_velo_to_cam = np.reshape(rawdata['Tr_velo_to_cam'], (3,4))

        data['P0'] = P0
        data['P1'] = P1
        data['P2'] = P2
        data['P3'] = P3
        data['R0_rect'] = R0_rect
        data['Tr_velo_to_cam'] = Tr_velo_to_cam

        return data

    def _read_calib_file(self, filepath):
        """Read in a calibration file and parse into a dictionary."""
        data = {}

        with open(filepath, 'r') as f:
            for line in f.readlines():
                key, value = line.split(':', 1)
                # The only non-float values in these files are dates, which
                # we don't care about anyway
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
        return data


class kittidataset(BaseDataset):
    """dataloader for kitti dataset"""
    @staticmethod
    def modify_commandline_options(parser, is_train):
        return parser

    def initialize(self, opt):
        self.opt = opt
        self.batch_size = opt.batch_size
        self.root = opt.dataroot # path for the dataset
        self.use_sne = opt.use_sne
        self.num_labels = 2
        self.use_size = (opt.useWidth, opt.useHeight)
        if self.use_sne:
            self.sne_model = SNE()

        if opt.phase == "train":
            self.image_list = sorted(glob.glob(os.path.join(self.root, 'training', 'image_2', '*.png')))
        elif opt.phase == "val":
            self.image_list = sorted(glob.glob(os.path.join(self.root, 'validation', 'image_2', '*.png')))
        else:
            self.image_list = sorted(glob.glob(os.path.join(self.root, 'testing', 'image_2', '*.png')))

    def __getitem__(self, index):
        useDir = "/".join(self.image_list[index].split('/')[:-2])
        name = self.image_list[index].split('/')[-1]
        path = os.path.join(useDir, 'image_2', name).replace("\'", '/')
        rgb_image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        depth_image = cv2.imread(path.replace("image_2","depth_u16"), cv2.IMREAD_ANYDEPTH)
        oriHeight, oriWidth, _ = rgb_image.shape

        if self.opt.phase == 'test' and self.opt.no_label:
            # Since we have no gt label (e.g., kitti submission), we generate pseudo gt labels to
            # avoid destroying the code architecture
            label = np.zeros((oriHeight, oriWidth), dtype=np.uint8)
        else:
            # path = os.path.join(useDir, 'gt_image_2', name[:-10]+'road_'+name[-10:])
            path = name.replace("image_2","gt_image_2").replace("um_","um_lane_")
            # print(path)
            label_image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
            label = np.zeros((oriHeight, oriWidth), dtype=np.uint8)
            label[label_image[:,:,2] > 0] = 1

        # resize image to enable sizes divide 32
        rgb_image = cv2.resize(rgb_image, self.use_size)
        label = cv2.resize(label, self.use_size, interpolation=cv2.INTER_NEAREST)

        # another_image will be normal when using SNE, otherwise will be depth
        if self.use_sne:
            calib = kittiCalibInfo(os.path.join(useDir, 'calib', name[:-4]+'.txt'))
            camParam = torch.tensor(calib.get_cam_param(), dtype=torch.float32)
            normal = self.sne_model(torch.tensor(depth_image.astype(np.float32)/1000), camParam)
            another_image = normal.cpu().numpy()
            another_image = np.transpose(another_image, [1, 2, 0])
            another_image = cv2.resize(another_image, self.use_size)
        else:
            another_image = depth_image.astype(np.float32)/65535
            another_image = cv2.resize(another_image, self.use_size)
            another_image = another_image[:,:,np.newaxis]

        label[label > 0] = 1
        rgb_image = rgb_image.astype(np.float32) / 255

        rgb_image = transforms.ToTensor()(rgb_image)
        another_image = transforms.ToTensor()(another_image)

        label = torch.from_numpy(label)
        label = label.type(torch.LongTensor)

        # return a dictionary containing useful information
        # input rgb images, another images and labels for training;
        # 'path': image name for saving predictions
        # 'oriSize': original image size for evaluating and saving predictions
        return {'rgb_image': rgb_image, 'another_image': another_image, 'label': label,
                'path': name, 'oriSize': (oriWidth, oriHeight)}

    def __len__(self):
        return len(self.image_list)

    def name(self):
        return 'kitti'

In [7]:

dataset = kittidataset()
train_opt.dataroot = "D:\Project Bipot\SNE-RoadSeg\datasets\kitti"
train_opt.phase = 'train'
dataset.initialize(train_opt)
result = dataset.__getitem__(0)
dataset.image_list = dataset.image_list[:24]
dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=1,
            )

In [8]:
train_opt.gpu_ids = [0]

In [9]:
train_opt = opt
train_opt.continue_train = False
train_opt.isTrain = True
train_opt.verbose = True
model = create_model(train_opt, dataset)
model.setup(train_opt)
total_steps = 0
tfcount = 0
F_score_max = 0

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Abhishek pandir/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:12<00:00, 3.74MB/s]


initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with 

In [10]:

from tensorboardX import SummaryWriter
writer = SummaryWriter()


In [ ]:
if True:
    for epoch in range(train_opt.epoch_count, train_opt.nepoch + 1):
        ### Training on the training set ###
        model.train()
        epoch_start_time = time.time()
        iter_data_time = time.time()
        epoch_iter = 0
        train_loss_iter = []
        for i, data in enumerate(dataloader):
            iter_start_time = time.time()
            if total_steps % train_opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time
            total_steps += train_opt.batch_size
            epoch_iter += train_opt.batch_size
            model.set_input(data)
            model.optimize_parameters()

            if total_steps % train_opt.print_freq == 0:
                tfcount = tfcount + 1
                losses = model.get_current_losses()
                train_loss_iter.append(losses["segmentation"])
                t = (time.time() - iter_start_time) / train_opt.batch_size
                print_current_losses(epoch, epoch_iter, losses, t, t_data)
                # There are several whole_loss values shown in tensorboard in one epoch,
                # to help better see the optimization phase
                writer.add_scalar('train/whole_loss', losses["segmentation"], tfcount)

            iter_data_time = time.time()

        mean_loss = np.mean(train_loss_iter)
        # One average training loss value in tensorboard in one epoch
        writer.add_scalar('train/mean_loss', mean_loss, epoch)

        palet_file = 'datasets/palette.txt'
        impalette = list(np.genfromtxt(palet_file,dtype=np.uint8).reshape(3*256))
        tempDict = model.get_current_visuals()
        rgb = tensor2im(tempDict['rgb_image'])
        if train_opt.use_sne:
            another = tensor2im((tempDict['another_image']+1)/2)    # color normal images
        else:
            another = tensor2im(tempDict['another_image'])
        label = tensor2labelim(tempDict['label'], impalette)
        output = tensor2labelim(tempDict['output'], impalette)
        image_numpy = np.concatenate((rgb, another, label, output), axis=1)
        image_numpy = image_numpy.astype(np.float64) / 255
        writer.add_image('Epoch' + str(epoch), image_numpy, dataformats='HWC')  # show training images in tensorboard

        print('End of epoch %d / %d \t Time Taken: %d sec' %   (epoch, train_opt.nepoch, time.time() - epoch_start_time))
        model.update_learning_rate()

        # ### Evaluation on the validation set ###
        # model.eval()
        # valid_loss_iter = []
        # epoch_iter = 0
        # conf_mat = np.zeros((valid_dataset.dataset.num_labels, valid_dataset.dataset.num_labels), dtype=np.float)
        # with torch.no_grad():
        #     for i, data in enumerate(valid_dataset):
        #         model.set_input(data)
        #         model.forward()
        #         model.get_loss()
        #         epoch_iter += valid_opt.batch_size
        #         gt = model.label.cpu().int().numpy()
        #         _, pred = torch.max(model.output.data.cpu(), 1)
        #         pred = pred.float().detach().int().numpy()

        #         # Resize images to the original size for evaluation
        #         image_size = model.get_image_oriSize()
        #         oriSize = (image_size[0].item(), image_size[1].item())
        #         gt = np.expand_dims(cv2.resize(np.squeeze(gt, axis=0), oriSize, interpolation=cv2.INTER_NEAREST), axis=0)
        #         pred = np.expand_dims(cv2.resize(np.squeeze(pred, axis=0), oriSize, interpolation=cv2.INTER_NEAREST), axis=0)

        #         conf_mat += confusion_matrix(gt, pred, valid_dataset.dataset.num_labels)
        #         losses = model.get_current_losses()
        #         valid_loss_iter.append(model.loss_segmentation)
        #         print('valid epoch {0:}, iters: {1:}/{2:} '.format(epoch, epoch_iter, len(valid_dataset) * valid_opt.batch_size), end='\r')

        # avg_valid_loss = torch.mean(torch.stack(valid_loss_iter))
        # globalacc, pre, recall, F_score, iou = getScores(conf_mat)

        # # Record performance on the validation set
        # writer.add_scalar('valid/loss', avg_valid_loss, epoch)
        # writer.add_scalar('valid/global_acc', globalacc, epoch)
        # writer.add_scalar('valid/pre', pre, epoch)
        # writer.add_scalar('valid/recall', recall, epoch)
        # writer.add_scalar('valid/F_score', F_score, epoch)
        # writer.add_scalar('valid/iou', iou, epoch)

        # # Save the best model according to the F-score, and record corresponding epoch number in tensorboard
        # if F_score > F_score_max:
        #     print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))
        #     model.save_networks('best')
        #     F_score_max = F_score
        #     writer.add_text('best model', str(epoch))


In [16]:
dataset = kittidataset()
train_opt.dataroot = "D:\Project Bipot\SNE-RoadSeg\datasets\kitti"
train_opt.phase = 'train'
dataset.initialize(train_opt)
result = dataset.__getitem__(0)
dataset.image_list = dataset.image_list[:24]
dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=1,
            )

In [13]:
import time
from options.train_options import TrainOptions
from data import CreateDataLoader
from models import create_model
from util.util import confusion_matrix, getScores, tensor2labelim, tensor2im, print_current_losses
import numpy as np
import random
import torch
import cv2
from tensorboardX import SummaryWriter
if True:    
    train_opt = opt

    np.random.seed(train_opt.seed)
    random.seed(train_opt.seed)
    torch.manual_seed(train_opt.seed)
    torch.cuda.manual_seed(train_opt.seed)

    train_opt.nepoch = 5
    train_opt.phase = 'train'
    train_opt.batch_size =1
    train_data_loader = CreateDataLoader(train_opt)
    train_dataset = train_data_loader.load_data()
    train_dataset_size = len(train_data_loader)
    print('#training images = %d' % train_dataset_size)

dataset [kitti] was created
#training images = 289


In [14]:

    writer = SummaryWriter()

    model = create_model(train_opt, train_dataset.dataset)
    model.setup(train_opt)
    total_steps = 0
    tfcount = 0
    F_score_max = 0

initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with pretrained
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with kaiming
initialize network with 

In [17]:
if __name__ == '__main__':
    for epoch in range(train_opt.epoch_count, train_opt.nepoch + 1):
        ### Training on the training set ###
        model.train()
        epoch_start_time = time.time()
        iter_data_time = time.time()
        epoch_iter = 0
        train_loss_iter = []
        for i, data in enumerate(dataloader):
            iter_start_time = time.time()
            if total_steps % train_opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time
            total_steps += train_opt.batch_size
            epoch_iter += train_opt.batch_size
            model.set_input(data)
            model.optimize_parameters()

            if total_steps % train_opt.print_freq == 0:
                tfcount = tfcount + 1
                losses = model.get_current_losses()
                train_loss_iter.append(losses["segmentation"])
                t = (time.time() - iter_start_time) / train_opt.batch_size
                print_current_losses(epoch, epoch_iter, losses, t, t_data)
                # There are several whole_loss values shown in tensorboard in one epoch,
                # to help better see the optimization phase
                writer.add_scalar('train/whole_loss', losses["segmentation"], tfcount)

            iter_data_time = time.time()

        mean_loss = np.mean(train_loss_iter)
        # One average training loss value in tensorboard in one epoch
        writer.add_scalar('train/mean_loss', mean_loss, epoch)

        palet_file = 'datasets/palette.txt'
        impalette = list(np.genfromtxt(palet_file,dtype=np.uint8).reshape(3*256))
        tempDict = model.get_current_visuals()
        rgb = tensor2im(tempDict['rgb_image'])
        if train_opt.use_sne:
            another = tensor2im((tempDict['another_image']+1)/2)    # color normal images
        else:
            another = tensor2im(tempDict['another_image'])
        label = tensor2labelim(tempDict['label'], impalette)
        output = tensor2labelim(tempDict['output'], impalette)
        image_numpy = np.concatenate((rgb, another, label, output), axis=1)
        image_numpy = image_numpy.astype(np.float64) / 255
        writer.add_image('Epoch' + str(epoch), image_numpy, dataformats='HWC')  # show training images in tensorboard

        print('End of epoch %d / %d \t Time Taken: %d sec' %   (epoch, train_opt.nepoch, time.time() - epoch_start_time))
        model.update_learning_rate()

        # ### Evaluation on the validation set ###
        # model.eval()
        # valid_loss_iter = []
        # epoch_iter = 0
        # conf_mat = np.zeros((valid_dataset.dataset.num_labels, valid_dataset.dataset.num_labels), dtype=np.float)
        # with torch.no_grad():
        #     for i, data in enumerate(valid_dataset):
        #         model.set_input(data)
        #         model.forward()
        #         model.get_loss()
        #         epoch_iter += valid_opt.batch_size
        #         gt = model.label.cpu().int().numpy()
        #         _, pred = torch.max(model.output.data.cpu(), 1)
        #         pred = pred.float().detach().int().numpy()

        #         # Resize images to the original size for evaluation
        #         image_size = model.get_image_oriSize()
        #         oriSize = (image_size[0].item(), image_size[1].item())
        #         gt = np.expand_dims(cv2.resize(np.squeeze(gt, axis=0), oriSize, interpolation=cv2.INTER_NEAREST), axis=0)
        #         pred = np.expand_dims(cv2.resize(np.squeeze(pred, axis=0), oriSize, interpolation=cv2.INTER_NEAREST), axis=0)

        #         conf_mat += confusion_matrix(gt, pred, valid_dataset.dataset.num_labels)
        #         losses = model.get_current_losses()
        #         valid_loss_iter.append(model.loss_segmentation)
        #         print('valid epoch {0:}, iters: {1:}/{2:} '.format(epoch, epoch_iter, len(valid_dataset) * valid_opt.batch_size), end='\r')

        # avg_valid_loss = torch.mean(torch.stack(valid_loss_iter))
        # globalacc, pre, recall, F_score, iou = getScores(conf_mat)

        # # Record performance on the validation set
        # writer.add_scalar('valid/loss', avg_valid_loss, epoch)
        # writer.add_scalar('valid/global_acc', globalacc, epoch)
        # writer.add_scalar('valid/pre', pre, epoch)
        # writer.add_scalar('valid/recall', recall, epoch)
        # writer.add_scalar('valid/F_score', F_score, epoch)
        # writer.add_scalar('valid/iou', iou, epoch)

        # # Save the best model according to the F-score, and record corresponding epoch number in tensorboard
        # if F_score > F_score_max:
        #     print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))
        #     model.save_networks('best')
        #     F_score_max = F_score
        #     writer.add_text('best model', str(epoch))


RuntimeError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 3.00 GiB total capacity; 1.87 GiB already allocated; 3.91 MiB free; 1.92 GiB reserved in total by PyTorch)